# **Vectorstores**


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from rich import print

os.environ['COHERE_API_KEY'] = os.getenv('COHERE_API_KEY')

## 1. FAISS - Facebook AI Similarity Search

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_community.vectorstores import  FAISS

# load the document
loader = TextLoader("Data Transformation Simplified Expl.txt")
document = loader.load()
# split the document into chunks
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=60)
doc_chunks = text_splitter.split_documents(document)

In [3]:
doc_chunks

[Document(metadata={'source': 'Data Transformation Simplified Expl.txt'}, page_content="Data Transformation: Simplified Explanation\n\nWhat is Data Transformation?\nData transformation is like changing the way we measure things to make them easier to understand and compare. It's like converting inches to centimeters or changing dollars to euros to help with calculations and comparisons."),
 Document(metadata={'source': 'Data Transformation Simplified Expl.txt'}, page_content='Why and Where?\nData transformation is important because it helps us work with data more effectively. We transform data to make it more useful for analysis and building models.'),
 Document(metadata={'source': 'Data Transformation Simplified Expl.txt'}, page_content='Practical Uses:\n- Scaling: Changing the range or size of numbers.\n- Normalization: Making data follow a certain pattern or shape.'),
 Document(metadata={'source': 'Data Transformation Simplified Expl.txt'}, page_content='Real-Life Example:\nThink ab

In [4]:
# we're using cohere embeddings
embeddings = CohereEmbeddings()
# create the vector store
vectorstore = FAISS.from_documents(doc_chunks, embeddings)
vectorstore

#### Search Method

In [5]:
# searching a word from vectorstore
retrieved_docs = vectorstore.search("Scaling", search_type="similarity", k=2)
print(retrieved_docs)

[
    Document(
        metadata={'source': 'Data Transformation Simplified Expl.txt'},
        page_content='Real-Life Example:\nThink about temperatures. In some places, they use Celsius, and in 
others, Fahrenheit. If you want to compare temperatures accurately, you might convert everything to one scale, like
Celsius.\n\nMin-Max Scaling: Simplified Explanation'
    ),
    Document(
        metadata={'source': 'Data Transformation Simplified Expl.txt'},
        page_content="Real-Life Example:\nSuppose you're analyzing test scores from different subjects. Some 
subjects might have higher averages and more spread-out scores. To compare them fairly, you'd use Standard Scaling 
to bring them all to the same baseline."
    )
]

#### Similarity Search Method

In [6]:
# using similarity search
retrieved_docs = vectorstore.similarity_search("What is Data Transformation?", k=3)
print(retrieved_docs[0].page_content)

Data Transformation: Simplified Explanation

What is Data Transformation?
Data transformation is like changing the way we measure things to make them easier to understand and compare. It's 
like converting inches to centimeters or changing dollars to euros to help with calculations and comparisons.

#### Similarity Search Score Method

In [7]:
# using similarity search with score to retrieve relevant chunks. The lower the score, the more relevant. it uses L2 score 
# that is also called Manhatton Distance (it gives distance from query)
retrieved_docs = vectorstore.similarity_search_with_score("What is Data Transformation?", k=2)
print(retrieved_docs)

[
    (
        Document(
            metadata={'source': 'Data Transformation Simplified Expl.txt'},
            page_content="Data Transformation: Simplified Explanation\n\nWhat is Data Transformation?\nData 
transformation is like changing the way we measure things to make them easier to understand and compare. It's like 
converting inches to centimeters or changing dollars to euros to help with calculations and comparisons."
        ),
        2909.331
    ),
    (
        Document(
            metadata={'source': 'Data Transformation Simplified Expl.txt'},
            page_content='Why and Where?\nData transformation is important because it helps us work with data more 
effectively. We transform data to make it more useful for analysis and building models.'
        ),
        3731.0398
    )
]

#### Similarity Search by Vector Method 
We can also pass a vector instead of text query to vectorstore.


In [8]:
vector_query = embeddings.embed_query("What is Data Transformation?")
vector_retrieved_docs = vectorstore.similarity_search_by_vector(vector_query, k=2)
print(vector_retrieved_docs)

[
    Document(
        metadata={'source': 'Data Transformation Simplified Expl.txt'},
        page_content="Data Transformation: Simplified Explanation\n\nWhat is Data Transformation?\nData 
transformation is like changing the way we measure things to make them easier to understand and compare. It's like 
converting inches to centimeters or changing dollars to euros to help with calculations and comparisons."
    ),
    Document(
        metadata={'source': 'Data Transformation Simplified Expl.txt'},
        page_content='Why and Where?\nData transformation is important because it helps us work with data more 
effectively. We transform data to make it more useful for analysis and building models.'
    )
]

### Saving Vectorstore in Local and then Loading it

In [9]:
vectorstore.save_local("faiss_index") # pickel file will be saved in current directory

In [10]:
# loading vectorstore
new_vectorstore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
new_vectorstore

In [11]:
# retrieving chunks from our loaded vectorstore
chunks = new_vectorstore.similarity_search_with_score("What is Min-Max Scaling?", k=2) 
print(chunks)

[
    (
        Document(
            metadata={'source': 'Data Transformation Simplified Expl.txt'},
            page_content="Min-Max Scaling: Simplified Explanation\n\nStandard Scaling is like making all your 
numbers play nicely with each other by giving them a common center (average) and making sure they're all about the 
same distance (standard deviation) from that center."
        ),
        2884.753
    ),
    (
        Document(
            metadata={'source': 'Data Transformation Simplified Expl.txt'},
            page_content='Real-Life Example:\nThink about temperatures. In some places, they use Celsius, and in 
others, Fahrenheit. If you want to compare temperatures accurately, you might convert everything to one scale, like
Celsius.\n\nMin-Max Scaling: Simplified Explanation'
        ),
        4505.025
    )
]

## Vectorstore as a Retriever

In [12]:
retriever = vectorstore.as_retriever()
retriever.invoke("What is Data Transformation?")

[Document(metadata={'source': 'Data Transformation Simplified Expl.txt'}, page_content="Data Transformation: Simplified Explanation\n\nWhat is Data Transformation?\nData transformation is like changing the way we measure things to make them easier to understand and compare. It's like converting inches to centimeters or changing dollars to euros to help with calculations and comparisons."),
 Document(metadata={'source': 'Data Transformation Simplified Expl.txt'}, page_content='Why and Where?\nData transformation is important because it helps us work with data more effectively. We transform data to make it more useful for analysis and building models.'),
 Document(metadata={'source': 'Data Transformation Simplified Expl.txt'}, page_content="In summary, data transformation methods like Min-Max Scaling and Standard Scaling are like adjusting data to make it easier to work with and compare. They're useful for various situations where you want to ensure fair comparisons and prevent some data

## 2. Chroma

In [13]:
from langchain_community.vectorstores import Chroma

chroma_vectorstore = Chroma.from_documents(doc_chunks, embeddings, persist_directory="./chroma_db")
chroma_vectorstore

#### Similarity Search Method

In [14]:
retrieved_docs = chroma_vectorstore.similarity_search("What is Data Transformation?", k=2)
print(retrieved_docs)

[
    Document(
        metadata={'source': 'Data Transformation Simplified Expl.txt'},
        page_content="Data Transformation: Simplified Explanation\n\nWhat is Data Transformation?\nData 
transformation is like changing the way we measure things to make them easier to understand and compare. It's like 
converting inches to centimeters or changing dollars to euros to help with calculations and comparisons."
    ),
    Document(
        metadata={'source': 'Data Transformation Simplified Expl.txt'},
        page_content='Why and Where?\nData transformation is important because it helps us work with data more 
effectively. We transform data to make it more useful for analysis and building models.'
    )
]

#### Similarity Search Score Method

In [15]:
retrieved_docs = chroma_vectorstore.similarity_search_with_score("What is Data Transformation?", k=2)
print(retrieved_docs)

[
    (
        Document(
            metadata={'source': 'Data Transformation Simplified Expl.txt'},
            page_content="Data Transformation: Simplified Explanation\n\nWhat is Data Transformation?\nData 
transformation is like changing the way we measure things to make them easier to understand and compare. It's like 
converting inches to centimeters or changing dollars to euros to help with calculations and comparisons."
        ),
        2908.829267868754
    ),
    (
        Document(
            metadata={'source': 'Data Transformation Simplified Expl.txt'},
            page_content='Why and Where?\nData transformation is important because it helps us work with data more 
effectively. We transform data to make it more useful for analysis and building models.'
        ),
        3730.245893735579
    )
]

### Loading Chroma DB and using it

In [16]:
new_chroma_db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)
new_chroma_db.similarity_search("What is Data Transformation?", k=2)

[Document(metadata={'source': 'Data Transformation Simplified Expl.txt'}, page_content="Data Transformation: Simplified Explanation\n\nWhat is Data Transformation?\nData transformation is like changing the way we measure things to make them easier to understand and compare. It's like converting inches to centimeters or changing dollars to euros to help with calculations and comparisons."),
 Document(metadata={'source': 'Data Transformation Simplified Expl.txt'}, page_content='Why and Where?\nData transformation is important because it helps us work with data more effectively. We transform data to make it more useful for analysis and building models.')]

#### Deleting ChromaDB from local

In [17]:
ids = chroma_vectorstore.get(where={'source': 'Data Transformation Simplified Expl.txt'})['ids']
ids

['24d08828-0d66-49a3-84be-b75abc172865',
 '316f3c83-cee3-4fa0-b6cb-7fc5ae80bb6f',
 '3fdebdd0-c9ff-4566-8169-371796ef0b6c',
 '41242368-0614-4231-a30f-cf53c99c9e14',
 '854966c9-840d-4340-b522-c879c9947e06',
 '9716dd0f-e6fe-40f3-bb0e-3b273248c13b',
 'a221fc06-4407-412b-b5f1-ea21c5a8ac39',
 'c7f1e0f4-4f8a-4f81-b5d2-92e7f4dcc21e',
 'c801e448-52ce-4378-b979-096fe26802e9']

In [18]:
# deleting all ids
# chroma_vectorstore.delete(ids=ids)

## Vectorstore as a Retriever (Chroma)

In [19]:
retriever = new_chroma_db.as_retriever()
retriever.invoke("What is Min-Max Scaling?")

[Document(metadata={'source': 'Data Transformation Simplified Expl.txt'}, page_content="Min-Max Scaling: Simplified Explanation\n\nStandard Scaling is like making all your numbers play nicely with each other by giving them a common center (average) and making sure they're all about the same distance (standard deviation) from that center."),
 Document(metadata={'source': 'Data Transformation Simplified Expl.txt'}, page_content='Real-Life Example:\nThink about temperatures. In some places, they use Celsius, and in others, Fahrenheit. If you want to compare temperatures accurately, you might convert everything to one scale, like Celsius.\n\nMin-Max Scaling: Simplified Explanation'),
 Document(metadata={'source': 'Data Transformation Simplified Expl.txt'}, page_content="In summary, data transformation methods like Min-Max Scaling and Standard Scaling are like adjusting data to make it easier to work with and compare. They're useful for various situations where you want to ensure fair compa

## 3. Quadrant

In [30]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

qdrant_key = "X3Y-R-zJaLG7JWsL_6hPNk2kTnehzvbn9DRoez5K2xKOaEhcloeWGg"
URL = "https://e8729f8b-0070-461b-9b27-55fec3c3af4a.us-east4-0.gcp.cloud.qdrant.io"

# creating qdrant vectorstore
qdrant = QdrantVectorStore.from_documents(
        doc_chunks,
        embeddings,
        url=URL,
        prefer_grpc=False,
        api_key=qdrant_key,
        collection_name = "text_collection",
        force_recreate = True
)
# 
qdrant_client = QdrantClient(
url=URL,
api_key=qdrant_key,
)

# getting the vectorstore. it's like we previously loaded Chroma
qdrant_store = QdrantVectorStore(qdrant_client, "text_collection" ,embeddings)
qdrant_store


In [31]:
print(qdrant_store.similarity_search("What is Data Transformation?"))

[
    Document(
        metadata={
            'source': 'Data Transformation Simplified Expl.txt',
            '_id': 'fd312940-302a-4e77-8253-308c672fa8be',
            '_collection_name': 'text_collection'
        },
        page_content="Data Transformation: Simplified Explanation\n\nWhat is Data Transformation?\nData 
transformation is like changing the way we measure things to make them easier to understand and compare. It's like 
converting inches to centimeters or changing dollars to euros to help with calculations and comparisons."
    ),
    Document(
        metadata={
            'source': 'Data Transformation Simplified Expl.txt',
            '_id': '1feff27f-eb06-46a8-b288-6a3577473945',
            '_collection_name': 'text_collection'
        },
        page_content='Why and Where?\nData transformation is important because it helps us work with data more 
effectively. We transform data to make it more useful for analysis and building models.'
    ),
    Document(
        metadata={
            'source': 'Data Transformation Simplified Expl.txt',
            '_id': '3c290a38-3767-4500-a9b3-c6a96c00417d',
            '_collection_name': 'text_collection'
        },
        page_content="In summary, data transformation methods like Min-Max Scaling and Standard Scaling are like 
adjusting data to make it easier to work with and compare. They're useful for various situations where you want to 
ensure fair comparisons and prevent some data from overshadowing others. Choosing the right method depends on what 
you're trying to achieve and the nature of your data."
    ),
    Document(
        metadata={
            'source': 'Data Transformation Simplified Expl.txt',
            '_id': '010b6f9c-a5f7-45e8-b884-e4bb3488a2a6',
            '_collection_name': 'text_collection'
        },
        page_content='Practical Uses:\n- Scaling: Changing the range or size of numbers.\n- Normalization: Making 
data follow a certain pattern or shape.'
    )
]